In [3]:
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
data = pd.read_csv("donors.csv", header=None, sep=',')

/Users/miguelcoutinhonunes/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3,9,21,22,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,51,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,27

In [5]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,466,467,468,469,470,471,472,473,474,475
0,NaN,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
1,0.0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,...,7.7419354839,95515,0,L,4,E,X,X,X,C
2,1.0,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,...,15.666666667000001,148535,0,L,2,G,X,X,X,A
3,2.0,2010-01-01,AMH,1,NC,27017,,,NaN,0,...,7.4814814814999995,15078,1,L,4,E,X,X,X,C
4,3.0,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,...,6.8125,172556,1,L,4,E,X,X,X,C


In [6]:
data.columns

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            466, 467, 468, 469, 470, 471, 472, 473, 474, 475],
           dtype='int64', length=476)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95413 entries, 0 to 95412
Columns: 476 entries, 0 to 475
dtypes: float64(1), object(475)
memory usage: 346.5+ MB


In [9]:
data.shape

(95413, 476)